In [1]:
from copy import copy
import math
import os
from pathlib import Path
import random
import shutil
import sys
sys.path.insert(0,'..')
from matplotlib import cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
from openstudio import isomodel
import pandas as pd
import seaborn as sns
from doe_xstock.database import SQLiteDatabase
from doe_xstock.simulate import OpenStudioModelEditor
from doe_xstock.utilities import write_data

In [2]:
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0
DATABASE_FILEPATH = '../database.db'
DATABASE = SQLiteDatabase(DATABASE_FILEPATH)

# Validate E+ results and select buildings with valid results based on temperature-setpoint difference

### Comparing ideal load temperature and setpoint

In [33]:
# select buildings where ideal loads are indeed being met
limit = 0.5
data_list = []

neighborhoods = {
    'tx_travis': (3625, 5832),
    'ca_alameda': (1, 2088),
    'vt_chittenden': (1, 2088)
}

for n in neighborhoods:
    filepath = os.path.join(Path('../data/neighborhood'), f'{n}_county_neighborhood.csv')
    n_data = pd.read_csv(filepath, sep='|')
    metadata_ids = n_data['metadata_id'].tolist()
    metadata_ids = tuple(metadata_ids)
    start_timestep, end_timestep = neighborhoods[n]
    data = DATABASE.query_table(f"""
    SELECT
        m.bldg_id,
        w.timestep,
        w.average_indoor_air_temperature - e.setpoint AS value
    FROM (SELECT * FROM energyplus_simulation WHERE reference = 1 AND metadata_id IN {metadata_ids}) t
    LEFT JOIN metadata m ON
        m.id = t.metadata_id
    LEFT JOIN energyplus_ideal_system_simulation w ON
        w.simulation_id = t.id
    LEFT JOIN ecobee_timeseries e ON 
        e.building_id = t.ecobee_building_id AND e.timestep = w.timestep
    ;
    """)

    # HEATMAP
    plot_data = data.copy()
    plot_data.loc[plot_data['value'].abs() < limit,'value'] = None
    plot_data['county'] = n
    plot_data = plot_data.reset_index()
    data_list.append(plot_data)
    plot_data = plot_data.pivot(index='bldg_id',columns='timestep',values='value')
    plot_data.index = plot_data.index.astype(str)

    # y ordering
    y_order = plot_data.sum(axis=1).sort_values()
    y_order = pd.DataFrame(y_order)
    y_order['order'] = y_order.reset_index().index
    plot_data = plot_data.merge(y_order[['order']],how='left',left_index=True,right_index=True)
    plot_data = plot_data.sort_values('order').drop(columns=['order'])

    x, y, z = plot_data.columns.tolist(), plot_data.index.tolist(), plot_data.values
    cmap = 'coolwarm'
    norm = colors.TwoSlopeNorm(vcenter=0)
    fig, ax = plt.subplots(1,1,figsize=(15,10))
    pcm = ax.pcolormesh(x,y,z,shading='nearest',norm=norm,cmap=cmap,edgecolors='black',linewidth=0.0)
    _ = fig.colorbar(pcm,ax=ax,orientation='vertical',label=r'$T_{ideal} - T_{setpoint}$ (C)',fraction=0.015,pad=0.01)
    ax.tick_params('x',which='both',rotation=0)
    ax.set_xlabel('Timestep')
    ax.set_ylabel('Building ID')
    ax.set_title(f'{n}; timesteps = ({start_timestep}, {end_timestep}); limit = {limit} C')
    filename = f'{n.lower().replace(" ", "_").replace(",", "")}_and_ideal_and_setpoint_temperature_difference.png'
    plt.savefig(os.path.join('../figures', filename), bbox_inches='tight', transparent=False)
    plt.close()

temperature_difference_data = pd.concat(data_list, ignore_index=True)
del data_list

- Pretty much all of Travis buildings are meeting setpoint
- About 50% of Alameda buildings are NOT meeting set point
- Somewhere around 80% of Chittenden buildings are meeting setpoint
- Not too sure what the case is for situations where setpoint is not met ut guess is either the reconfiguration of the building EMS to allow for ideal and partial load simulation as well as the removal of mechanical systems could have messed things up. In some cases however, the setpoint is too extreme. An example is bldg_id = 146210 in Alameda whose setpoint is ~ 7.8C on average.


Need to find a sweet spot of buildings in each county where the compromise on unmet hours is not too severe:

In [34]:
limit = 5.0
temperature_difference_data['unmet'] = 0
temperature_difference_data.loc[temperature_difference_data['value'].notnull(), 'unmet'] = 1
plot_data = temperature_difference_data.groupby(['county', 'bldg_id'])[['unmet']].sum()
plot_data['percentage'] = plot_data['unmet']*100.0/8760
plot_data = plot_data[plot_data['percentage']<=limit].reset_index()
valid_buildings = plot_data.copy()
plot_data = plot_data.groupby('county').size().reset_index(name='count')
plot_data

,county,count
0,ca_alameda,79
1,tx_travis,100
2,vt_chittenden,91


### Comparing 100% partial (OtherEquipment object in E+) temperature to setpoint

In [3]:
limit = 0.5
data_list = []

neighborhoods = {
    'tx_travis': (3625, 5832),
    'ca_alameda': (1, 2088),
    'vt_chittenden': (1, 2088)
}

for n in neighborhoods:
    filepath = os.path.join(Path('../data/neighborhood'), f'{n}_county_neighborhood.csv')
    n_data = pd.read_csv(filepath, sep='|')
    metadata_ids = n_data['metadata_id'].tolist()
    metadata_ids = tuple(metadata_ids)
    start_timestep, end_timestep = neighborhoods[n]
    plot_data = DATABASE.query_table(f"""
    SELECT
        m.bldg_id,
        l.timestep,
        l.average_indoor_air_temperature - e.setpoint AS value
    FROM (SELECT * FROM energyplus_simulation WHERE reference = 2 AND metadata_id IN {metadata_ids}) t
    LEFT JOIN metadata m ON
        m.id = t.metadata_id
    LEFT JOIN lstm_train_data l ON
        l.simulation_id = t.id
    LEFT JOIN ecobee_timeseries e ON 
        e.building_id = t.ecobee_building_id AND e.timestep = l.timestep
    WHERE l.timestep BETWEEN {start_timestep} AND {end_timestep}
    ;
    """)
    
    # HEATMAP
    plot_data = plot_data.copy()
    plot_data.loc[plot_data['value'].abs() < limit,'value'] = None
    plot_data['county'] = n
    plot_data = plot_data.reset_index()
    data_list.append(plot_data)
    plot_data = plot_data.pivot(index='bldg_id',columns='timestep',values='value')
    plot_data.index = plot_data.index.astype(str)

    # y ordering
    y_order = plot_data.sum(axis=1).sort_values()
    y_order = pd.DataFrame(y_order)
    y_order['order'] = y_order.reset_index().index
    plot_data = plot_data.merge(y_order[['order']],how='left',left_index=True,right_index=True)
    plot_data = plot_data.sort_values('order').drop(columns=['order'])

    x, y, z = plot_data.columns.tolist(), plot_data.index.tolist(), plot_data.values
    cmap = 'coolwarm'
    norm = colors.TwoSlopeNorm(vcenter=0)
    fig, ax = plt.subplots(1,1,figsize=(15,10))
    pcm = ax.pcolormesh(x,y,z,shading='nearest',norm=norm,cmap=cmap,edgecolors='black',linewidth=0.0)
    _ = fig.colorbar(pcm,ax=ax,orientation='vertical',label=r'$T_{100 partial} - T_{setpoint}$ (C)',fraction=0.015,pad=0.01)
    ax.tick_params('x',which='both',rotation=0)
    ax.set_xlabel('Timestep')
    ax.set_ylabel('Building ID')
    ax.set_title(f'{n}; timesteps = ({start_timestep}, {end_timestep}); limit = {limit} C')
    filename = f'{n.lower().replace(" ", "_").replace(",", "")}_100p_and_setpoint_temperature_difference.png'
    plt.savefig(os.path.join('../figures', filename), bbox_inches='tight', transparent=False)
    plt.close()

temperature_difference_data = pd.concat(data_list, ignore_index=True)
del data_list

In [4]:
limit = 5.0
temperature_difference_data['unmet'] = 0
temperature_difference_data.loc[temperature_difference_data['value'].notnull(), 'unmet'] = 1
plot_data = temperature_difference_data.groupby(['county', 'bldg_id'])[['unmet']].sum()
plot_data['percentage'] = plot_data['unmet']*100.0/8760
plot_data = plot_data[plot_data['percentage']<=limit].reset_index()
valid_buildings = plot_data.copy()
plot_data = plot_data.groupby('county').size().reset_index(name='count')
plot_data

,county,count
0,ca_alameda,73
1,tx_travis,100
2,vt_chittenden,43


In [5]:
citylearn_buildings = {}

for c, c_data in valid_buildings.groupby('county'):
    citylearn_buildings[c] = c_data['bldg_id'].tolist()

# CityLearn Input Data

In [14]:
# building data
energyplus_output_directory = Path('../energyplus_output')
neighborhoods = [
    'tx_travis',
    'ca_alameda',
    'vt_chittenden'
]

for n in neighborhoods: 
    source_path = Path(f'../data/neighborhood/{n}_county_neighborhood.csv')
    destination_directory = Path(f'../citylearn_input/{n}_county_neighborhood')

    os.makedirs(destination_directory, exist_ok=True)
    source = pd.read_csv(source_path, sep='|')
    source = source[source['bldg_id'].isin(citylearn_buildings[n])].copy()
    query = """
    WITH u AS (
        -- weighted conditioned zone variables
        SELECT
            r.TimeIndex,
            r.ReportDataDictionaryIndex,
            'weighted_variable' AS label,
            r.Value
        FROM weighted_variable r
        LEFT JOIN ReportDataDictionary d ON d.ReportDataDictionaryIndex = r.ReportDataDictionaryIndex
        WHERE d.Name IN ('Zone Air Temperature', 'Zone Air Relative Humidity')

        UNION ALL

        -- thermal load variables
        SELECT
            r.TimeIndex,
            r.ReportDataDictionaryIndex,
            CASE WHEN r.Value > 0 THEN 'heating_load' ELSE 'cooling_load' END AS label,
            ABS(r.Value) AS Value
        FROM ReportData r
        LEFT JOIN ReportDataDictionary d ON d.ReportDataDictionaryIndex = r.ReportDataDictionaryIndex
        WHERE 
            d.Name = 'Other Equipment Convective Heating Rate' AND
            (d.KeyValue LIKE '%HEATING LOAD' OR d.KeyValue LIKE '%COOLING LOAD')

        UNION ALL

        -- other variables
        SELECT
            r.TimeIndex,
            r.ReportDataDictionaryIndex,
            'other' AS label,
            r.Value
        FROM ReportData r
        LEFT JOIN ReportDataDictionary d ON d.ReportDataDictionaryIndex = r.ReportDataDictionaryIndex
        WHERE 
            d.Name IN (
                'Water Heater Use Side Heat Transfer Energy',
                'Exterior Lights Electricity Energy',
                'Lights Electricity Energy',
                'Electric Equipment Electricity Energy',
                'Zone People Occupant Count'
            )
    ), p AS (
        SELECT
            u.TimeIndex,
            SUM(CASE WHEN d.Name = 'Zone Air Temperature' THEN Value END) AS "Indoor Temperature (C)",
            SUM(CASE WHEN d.Name = 'Zone Air Relative Humidity' THEN Value END) AS "Indoor Relative Humidity (%)",
            SUM(CASE WHEN d.Name IN ('Exterior Lights Electricity Energy', 'Lights Electricity Energy', 'Electric Equipment Electricity Energy') THEN Value/(3600.0*1000.0) END) AS "Equipment Electric Power (kWh)",
            SUM(CASE WHEN d.Name = 'Water Heater Use Side Heat Transfer Energy' THEN ABS(Value)/(3600.0*1000.0) END) AS "DHW Heating (kWh)",
            SUM(CASE WHEN u.label = 'cooling_load' THEN ABS(Value)/(1000.0) END) AS "Cooling Load (kWh)",
            SUM(CASE WHEN u.label = 'heating_load' THEN ABS(Value)/(1000.0) END) AS "Heating Load (kWh)",
            SUM(CASE WHEN d.Name = 'Zone People Occupant Count' THEN Value END) AS "Occupancy"
        FROM u
        LEFT JOIN ReportDataDictionary d ON d.ReportDataDictionaryIndex = u.ReportDataDictionaryIndex
        GROUP BY u.TimeIndex
    )

    SELECT
        t.Month AS "Month",
        t.Hour AS "Hour",
        CASE
            WHEN t.DayType = 'Monday' THEN 1
            WHEN t.DayType = 'Tuesday' THEN 2
            WHEN t.DayType = 'Wednesday' THEN 3
            WHEN t.DayType = 'Thursday' THEN 4
            WHEN t.DayType = 'Friday' THEN 5
            WHEN t.DayType = 'Saturday' THEN 6
            WHEN t.DayType = 'Sunday' THEN 7
            WHEN t.DayType = 'Holiday' THEN 8
            ELSE NULL
        END AS "Day Type",
        t.Dst AS "Daylight Savings Status",
        p."Indoor Temperature (C)",
        NULL AS "Average Unmet Cooling Setpoint Difference (C)",
        p."Indoor Relative Humidity (%)",
        p."Equipment Electric Power (kWh)",
        p."DHW Heating (kWh)",
        COALESCE(p."Cooling Load (kWh)", 0.0) AS "Cooling Load (kWh)",
        COALESCE(p."Heating Load (kWh)", 0.0) AS "Heating Load (kWh)",
        NULL AS "Solar Generation (W/kW)",
        p."Occupancy"
    FROM p
    LEFT JOIN Time t ON t.TimeIndex = p.TimeIndex
    WHERE t.DayType NOT IN ('SummerDesignDay', 'WinterDesignDay')
    ORDER BY t.TimeIndex
    """
    directories = sorted([
        d for d in os.listdir(energyplus_output_directory) 
        if d.endswith('2-partial') and int(d.split('-')[5]) in source['bldg_id'].to_list()
    ])

    for i, d in enumerate(directories):
        print(f'\r{i+1}/{len(directories)}', end=' '*30)
        bldg_id = d.split('-')[-3]

        if int(bldg_id) in source['bldg_id'].tolist():
            sim_id = d.replace('output_','')
            f = os.path.join(energyplus_output_directory, d, f'{sim_id}.sql')
            db = SQLiteDatabase(f)
            data = db.query_table(query)
            setpoint_data = pd.read_csv(f.replace('.sql', '_setpoint.csv'))
            data['Set Point'] =setpoint_data['setpoint']
            data.to_csv(os.path.join(destination_directory, f'{sim_id.split(bldg_id)[0]}{bldg_id}.csv'), index=False)
        
        else:
            continue

43/43                                

# Weather data

In [15]:
neighborhoods = {
    'tx_travis': '101010',
    'ca_alameda': '519230',
    'vt_chittenden': '276908',
}

for n, b in neighborhoods.items():
    print(n)
    simulation_id = f'resstock-amy2018-2021-release-1-{b}-0-mechanical'
    source_directory = Path(f'../energyplus_output/output_{simulation_id}/')
    f = os.path.join(source_directory, f'{simulation_id}.sql')
    destination_directory = Path(f'../citylearn_input/{n}_county_neighborhood')
    os.makedirs(destination_directory, exist_ok=True)
    shutil.copyfile(os.path.join(source_directory, f'weather.epw'), os.path.join(destination_directory, f'{n}.epw'))
    db = SQLiteDatabase(f)
    data = db.query_table("""
    SELECT
        MAX(CASE WHEN d.Name = 'Site Outdoor Air Drybulb Temperature' THEN r.value END) AS "Outdoor Drybulb Temperature (C)",
        MAX(CASE WHEN d.Name = 'Site Outdoor Air Relative Humidity' THEN r.value END) AS "Outdoor Relative Humidity (%)",
        MAX(CASE WHEN d.Name = 'Site Diffuse Solar Radiation Rate per Area' THEN r.value END) AS "Diffuse Solar Radiation (W/m2)",
        MAX(CASE WHEN d.Name = 'Site Direct Solar Radiation Rate per Area' THEN r.value END) AS "Direct Solar Radiation (W/m2)"
    FROM ReportData r
    LEFT JOIN ReportDataDictionary d ON d.ReportDataDictionaryIndex = r.ReportDataDictionaryIndex
    LEFT JOIN Time t ON t.TimeIndex = r.TimeIndex
    WHERE t.DayType NOT IN ('SummerDesignDay', 'WinterDesignDay')
    GROUP BY t.TimeIndex
    ORDER BY t.TimeIndex
    """)
    shifts = [6, 12, 24]
    accuracy = [0.025, 0.05, 0.1]
    columns = data.columns

    for c in columns:
        for s, a in zip(shifts, accuracy):
            arr = np.roll(data[c], shift=-s)
            random.seed(arr.mean())
            mini = 0 if arr.min() >= 0 else -a
            maxi = a
            data[f'{s}h {c}'] = arr + arr*np.random.uniform(mini, maxi, len(arr))

    data.to_csv(os.path.join(destination_directory, 'weather.csv'), index=False)
    

tx_travis
ca_alameda
vt_chittenden


# Estimate roof area

In [29]:
energyplus_output_directory = Path('../energyplus_output')
neighborhoods = [
    'tx_travis',
    'ca_alameda',
    'vt_chittenden'
]
trans = isomodel.ISOModelForwardTranslator()

for n in neighborhoods:
    data = []
    source_directory = Path(f'../citylearn_input/{n}_county_neighborhood')
    building_dircetories = [d for d in os.listdir(source_directory) if d.startswith('resstock')]

    for i, b in enumerate(building_dircetories):
        print(f'\r{n}: {i}/{len(building_dircetories)}', end=' '*40)
        simulation_id = b.split('.')[0]
        dataset_type, weather_data, year_of_publication, _, release, bldg_id = simulation_id.split('-')
        query = f"""
        SELECT 
            i.bldg_osm AS osm
        FROM energyplus_simulation_input i
        LEFT JOIN metadata m ON m.id = i.metadata_id
        WHERE 
            i.dataset_type = '{dataset_type}'
            AND i.dataset_weather_data = '{weather_data}'
            AND i.dataset_year_of_publication = {year_of_publication}
            AND i.dataset_release = {release}
            AND i.bldg_id = {bldg_id}
        """
        osm = DATABASE.query(query)[0][0][0]
        osm_editor = OpenStudioModelEditor(osm)
        osm_model = osm_editor.get_model()
        iso_model = trans.translateModel(osm_model)
        roof_area = iso_model.roofArea()
        data.append({'neighborhood': n, 'bldg_id': bldg_id, 'roof_area': roof_area})

    roof_area_data = pd.DataFrame(data)
    roof_area_data.to_csv(os.path.join(source_directory, 'roof_area.csv'), index=False)

vt_chittenden: 99/100                                        